# Robustness to Execution Perturbations

This notebook evaluates the scheduler's ability to recover from unexpected changes. Two perturbation types are applied during execution:

* **Gate injection** – extra entangling operations are inserted while the circuit is already running.
* **Cost corruption** – cost model coefficients are perturbed to trigger re-planning.

For each magnitude we perform multiple repetitions, measuring the time to recover and the runtime overhead. Timelines highlight when re-planning occurs and a table summarises recovery statistics.

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from quasar.circuit import Gate
from quasar.scheduler import Scheduler
from quasar.planner import Planner
from benchmarks import circuits

plt.rcParams['figure.dpi'] = 120

In [ ]:
# Baseline runtime

def measure_baseline():
    sched = Scheduler(planner=Planner())
    t0 = time.perf_counter()
    sched.run(circuits.ghz_circuit(5))
    return time.perf_counter() - t0

baseline_time = measure_baseline()
baseline_time

In [ ]:
# Perturbation helpers

def run_gate_perturbation(magnitude, repetitions=5):
    records = []
    for _ in range(repetitions):
        circuit = circuits.ghz_circuit(5)
        scheduler = Scheduler(planner=Planner())
        timeline = []
        info = {}
        step_idx = {'i': 0}
        def monitor(step, observed, est):
            now = time.perf_counter() - start
            start_t = timeline[-1]['end'] if timeline else 0.0
            timeline.append({'step': step_idx['i'], 'start': start_t, 'end': now, 'backend': step.backend.name})
            step_idx['i'] += 1
            if 'trigger' not in info:
                circuit.gates[step.end:step.end] = [Gate('CX', [0, 1])] * magnitude
                info['trigger'] = now
                return True
            return False
        start = time.perf_counter()
        scheduler.run(circuit, monitor=monitor)
        total = time.perf_counter() - start
        records.append({'perturb': 'gate', 'magnitude': magnitude, 'total_time': total,
                        'recovery_time': total - info['trigger'], 'overhead': total - baseline_time,
                        'timeline': timeline, 'trigger_time': info['trigger']})
    return records


def run_cost_perturbation(factor, repetitions=5):
    records = []
    for _ in range(repetitions):
        circuit = circuits.ghz_circuit(5)
        scheduler = Scheduler(planner=Planner())
        scheduler.planner.estimator.coeff['sv_gate'] *= factor
        timeline = []
        step_idx = {'i': 0}
        def monitor(step, observed, est):
            now = time.perf_counter() - start
            start_t = timeline[-1]['end'] if timeline else 0.0
            timeline.append({'step': step_idx['i'], 'start': start_t, 'end': now, 'backend': step.backend.name})
            step_idx['i'] += 1
            return False
        start = time.perf_counter()
        scheduler.run(circuit, monitor=monitor)
        total = time.perf_counter() - start
        trigger = timeline[0]['end'] if timeline else 0.0
        records.append({'perturb': 'cost', 'magnitude': factor, 'total_time': total,
                        'recovery_time': total - trigger, 'overhead': total - baseline_time,
                        'timeline': timeline, 'trigger_time': trigger})
    return records

In [ ]:
# Run experiments

repetitions = 5

gate_magnitudes = [1, 2, 3]
cost_magnitudes = [1e-6, 1e-7, 1e-8]

records = []
for m in gate_magnitudes:
    records.extend(run_gate_perturbation(m, repetitions))
for f in cost_magnitudes:
    records.extend(run_cost_perturbation(f, repetitions))

results = pd.DataFrame(records)
results.head()

In [ ]:
# Timeline plot for a representative run

sample = results[(results['perturb']=='gate') & (results['magnitude']==gate_magnitudes[0])].iloc[0]
fig, ax = plt.subplots(figsize=(6, 2))
for seg in sample['timeline']:
    ax.barh(seg['step'], seg['end']-seg['start'], left=seg['start'], color='tab:blue')
ax.axvline(sample['trigger_time'], color='red', linestyle='--', label='re-plan')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Step')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Summary table of recovery metrics

metrics = results.groupby(['perturb', 'magnitude']).agg(
    recovery_mean=('recovery_time', 'mean'),
    recovery_std=('recovery_time', 'std'),
    overhead_mean=('overhead', 'mean'),
    overhead_std=('overhead', 'std')
).reset_index()
metrics